In [1]:
filepath = 'C:\\districting-data-2010\\'

# 1. Code takes Daryl's block-level .json files and 
#    adds the lat/lon coordinates to the nodes
#    (based on the census shapefiles)
#
#outfilepath1 = 'C:\\districting-data-2020-lat-lon\\'

# 2. Then connects any disconnected graphs by adding
#    a minimum length subset of edges.
#
outfilepath2 = 'C:\\districting-data-2010-conn\\'

In [2]:
import os
if not os.path.exists(outfilepath2):
    os.makedirs(outfilepath2)

In [3]:
import geopandas as gpd
from gerrychain import Graph
import networkx as nx

In [4]:
state_codes = {
    'WA': '53', 'DE': '10', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

In [5]:
import json
from networkx.readwrite import json_graph

def write_graph_to_json(graph, json_file):
    data = json_graph.adjacency_data(graph)
    with open(json_file, "w") as f:
        json.dump(data,f)
    return

In [6]:
from math import cos, asin, sqrt, pi

# get approximate distance between lat/long coordinates:
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula?noredirect=1&lq=1
def apx_dist(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12.742 * asin(sqrt(a)) #2*R*asin...

In [7]:
for state in state_codes.keys():
    
    print("Beginning state",state)
    
    # read graph from file
    graphfile = state + '_block.json'
    G = Graph.from_json( filepath + graphfile )

#     # read the shapefile
#     shpfile = state + '_block.shp'
#     df = gpd.read_file( filepath + shpfile )
    
#     # add the lat/lon coordinates to graph (based on shapefile)
#     geoid_to_node = { G.nodes[i]['GEOID20'] : i for i in G.nodes }
#     for u in range(len(G.nodes)):
#         geoid = df['GEOID20'][u]
#         i = geoid_to_node[geoid]
#         G.nodes[i]['INTPTLAT20'] = float( df['INTPTLAT20'][u] )
#         G.nodes[i]['INTPTLON20'] = float( df['INTPTLON20'][u] )
        
#     # write the lat/lon-adjusted graph to output1
#     write_graph_to_json(G, outfilepath1 + graphfile)
    
    # is graph connected?
    if nx.is_connected(G):
        
        print("Graph is already connected.")
        
    else:
        
        # if not, then connect it with least-cost edges
        components = list(nx.connected_components(G))
        H = nx.complete_graph(len(components))

        for c in range(len(components)):
            component = components[c]
            print("Component",c,"has this many nodes:",len(component))

        for i,j in H.edges:

            min_dist = 999999999999.0 # some large number
            min_dist_edge = None

            for u in components[i]:

                lat1 = G.nodes[u]['C_Y']
                lon1 = G.nodes[u]['C_X']

                for v in components[j]:

                    lat2 = G.nodes[v]['C_Y']
                    lon2 = G.nodes[v]['C_X']

                    dist = apx_dist(lat1, lon1, lat2, lon2)

                    if dist < min_dist:

                        min_dist = dist
                        min_dist_edge = (u,v)

            H.edges[i,j]['weight'] = min_dist
            H.edges[i,j]['G_edge'] = min_dist_edge

        # find min span tree of H
        T = nx.minimum_spanning_tree(H,weight='weight')
        print("Spanning tree edges in H are =",T.edges)

        # which edges does this correspond to in G?
        edges_to_add = [ T.edges[i,j]['G_edge'] for i,j in T.edges ]
        print("Adding these edges to G =",edges_to_add)
        G.add_edges_from(edges_to_add)

        # print their GEOIDs for later reference
        added_edges_geoids = [ ( G.nodes[i]['geoid'], G.nodes[j]['geoid'] ) for i,j in edges_to_add ]
        print("Their GEOIDs are =",added_edges_geoids)

        # make sure G is now connected
        print("After adding these edges, is G now connected?",nx.is_connected(G))
        
    print("")
    # write the lat/lon-adjusted *and connected* graph to output2
    write_graph_to_json(G, outfilepath2 + graphfile)

Beginning state WA
Graph is already connected.

Beginning state DE
Graph is already connected.

Beginning state WI
Graph is already connected.

Beginning state WV
Graph is already connected.

Beginning state HI
Component 0 has this many nodes: 8888
Component 1 has this many nodes: 8819
Component 2 has this many nodes: 3
Component 3 has this many nodes: 18
Component 4 has this many nodes: 3
Component 5 has this many nodes: 16
Component 6 has this many nodes: 4
Component 7 has this many nodes: 4
Component 8 has this many nodes: 5
Component 9 has this many nodes: 3
Component 10 has this many nodes: 852
Component 11 has this many nodes: 2939
Component 12 has this many nodes: 31
Component 13 has this many nodes: 5
Component 14 has this many nodes: 3088
Component 15 has this many nodes: 338
Spanning tree edges in H are = [(0, 14), (1, 10), (1, 11), (2, 9), (2, 5), (3, 6), (3, 4), (4, 9), (5, 8), (6, 7), (7, 13), (10, 15), (11, 12), (12, 13), (14, 15)]
Adding these edges to G = [(3001, 23214)

C:\Users\buchanan\Anaconda3\lib\site-packages\gerrychain\graph\graph.py:239: UserWarning: Found islands (degree-0 nodes). Indices of islands: {162526, 163399}
  "Found islands (degree-0 nodes). Indices of islands: {}".format(islands)


Component 0 has this many nodes: 350167
Component 1 has this many nodes: 1
Component 2 has this many nodes: 1
Spanning tree edges in H are = [(0, 1), (1, 2)]
Adding these edges to G = [(163199, 162526), (162526, 163399)]
Their GEOIDs are = [('360610005000002', '360610001001000'), ('360610001001000', '360610001001001')]
After adding these edges, is G now connected? True

Beginning state PA
Graph is already connected.

Beginning state AK


C:\Users\buchanan\Anaconda3\lib\site-packages\gerrychain\graph\graph.py:239: UserWarning: Found islands (degree-0 nodes). Indices of islands: {1316, 1637, 1924, 1799, 584, 1321, 426, 1630, 2219, 9357, 22986, 29990, 29169, 29306, 1182, 8255}
  "Found islands (degree-0 nodes). Indices of islands: {}".format(islands)


Component 0 has this many nodes: 42381
Component 1 has this many nodes: 524
Component 2 has this many nodes: 45
Component 3 has this many nodes: 11
Component 4 has this many nodes: 1
Component 5 has this many nodes: 1
Component 6 has this many nodes: 2
Component 7 has this many nodes: 8
Component 8 has this many nodes: 60
Component 9 has this many nodes: 12
Component 10 has this many nodes: 446
Component 11 has this many nodes: 30
Component 12 has this many nodes: 5
Component 13 has this many nodes: 6
Component 14 has this many nodes: 29
Component 15 has this many nodes: 15
Component 16 has this many nodes: 70
Component 17 has this many nodes: 14
Component 18 has this many nodes: 2
Component 19 has this many nodes: 1
Component 20 has this many nodes: 34
Component 21 has this many nodes: 21
Component 22 has this many nodes: 1
Component 23 has this many nodes: 1
Component 24 has this many nodes: 10
Component 25 has this many nodes: 4
Component 26 has this many nodes: 6
Component 27 has t